# Data preprocessing

In [1]:
import pandas as pd
import numpy as np
from utils import check_missing_col_coverage

In [2]:
key = "SK_ID_CURR"

## 1. Presious application

In [3]:
prev_app = pd.read_csv("data/previous_application.csv")

prev_app.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
prev_app["NAME_CONTRACT_STATUS"].value_counts()

NAME_CONTRACT_STATUS
Approved        1036781
Canceled         316319
Refused          290678
Unused offer      26436
Name: count, dtype: int64

In [5]:
# AMT_ACCEPT_RATE: credit get compared to the amount the client applied
prev_app['AMT_ACCEPT_RATE'] = prev_app['AMT_CREDIT']/prev_app['AMT_APPLICATION']

In [6]:
prev_app_agg = {
    "NAME_CONTRACT_STATUS": ["count"],
    "AMT_ANNUITY": ["sum", "mean"],
    "AMT_APPLICATION": ["sum", "mean"]    
}

prev_app_status = prev_app[["NAME_CONTRACT_STATUS", "SK_ID_CURR", "SK_ID_PREV", "AMT_ANNUITY", "AMT_APPLICATION"]].groupby(["SK_ID_CURR","NAME_CONTRACT_STATUS"]).agg(prev_app_agg).reset_index()

prev_app_status.columns = [
    f"{col[0]}_{col[1].upper()}" if col[1] else col[0].upper()
    for col in prev_app_status.columns
]


pre_app_agg = prev_app_status.pivot_table(
        index="SK_ID_CURR",
        columns="NAME_CONTRACT_STATUS",
        values=["NAME_CONTRACT_STATUS_COUNT", "AMT_ANNUITY_SUM", "AMT_ANNUITY_MEAN", "AMT_APPLICATION_SUM", "AMT_APPLICATION_MEAN"],
        aggfunc="sum",     # combine if there are duplicates
        fill_value=0       # fill missing with 0
    ).reset_index()

pre_app_agg.columns = [
    f"{col[0]}_{col[1].upper()}" if col[1] else col[0].upper()
    for col in pre_app_agg.columns
]

# Save data to /etl folder
pre_app_agg.to_csv("etl/previous_applications_agg.csv", header=True, index=False)

## 2. Bureau

In [7]:
bu = pd.read_csv("data/bureau.csv")
bu.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [8]:
bu.CREDIT_ACTIVE.value_counts()

CREDIT_ACTIVE
Closed      1079273
Active       630607
Sold           6527
Bad debt         21
Name: count, dtype: int64

In [9]:
bu_app_agg = {
    "DAYS_CREDIT": ["min", "max", "mean"],
    "AMT_CREDIT_MAX_OVERDUE": ["max"],
    "AMT_CREDIT_SUM": ["sum", "mean"],
    "AMT_CREDIT_SUM_OVERDUE": ["min", "max", "mean"],
    "AMT_ANNUITY": ["sum", "mean"]
}

bu_status = bu.groupby(["SK_ID_CURR","CREDIT_ACTIVE"])[["DAYS_CREDIT", "AMT_CREDIT_MAX_OVERDUE", "AMT_CREDIT_SUM", "AMT_CREDIT_SUM_OVERDUE", "AMT_ANNUITY"]].agg(bu_app_agg).reset_index()

bu_status.columns = [
    f"{col[0]}_{col[1].upper()}" if col[1] else col[0].upper()
    for col in bu_status.columns
]

bu_agg = bu_status.pivot_table(
        index="SK_ID_CURR",
        columns="CREDIT_ACTIVE",
        values=['DAYS_CREDIT_MIN', 'DAYS_CREDIT_MAX',
       'DAYS_CREDIT_MEAN', 'AMT_CREDIT_MAX_OVERDUE_MAX', 'AMT_CREDIT_SUM_SUM',
       'AMT_CREDIT_SUM_MEAN', 'AMT_CREDIT_SUM_OVERDUE_MIN',
       'AMT_CREDIT_SUM_OVERDUE_MAX', 'AMT_CREDIT_SUM_OVERDUE_MEAN',
       'AMT_ANNUITY_SUM', 'AMT_ANNUITY_MEAN'],
        aggfunc="sum",     # combine if there are duplicates
        fill_value=0       # fill missing with 0
    ).reset_index()

bu_agg.columns = [
    f"{col[0]}_{col[1].upper()}" if col[1] else col[0].upper()
    for col in bu_agg.columns
]


# Save data to /etl folder
bu_agg.to_csv("etl/bureau_agg.csv", header=True, index=False)


## 3 Bureau Balance

## 4 POS_CASH

In [10]:
pos = pd.read_csv('data/POS_CASH_balance.csv')
pos.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [11]:
# Check missing vals
check_missing_col_coverage(pos)

CNT_INSTALMENT           0.260675
CNT_INSTALMENT_FUTURE    0.260835
dtype: float64

In [12]:
pos_aggmethod={'MONTHS_BALANCE':['min', 'max', 'count', 'mean'],
               'CNT_INSTALMENT':['min', 'max', 'mean'],
               'CNT_INSTALMENT_FUTURE':['min', 'max', 'mean'],
               'SK_DPD':['min', 'max', 'mean','sum'],
               'SK_DPD_DEF':['min', 'max', 'mean','sum']}

pos_agg = pos.groupby(["SK_ID_CURR", "NAME_CONTRACT_STATUS"]).agg(pos_aggmethod).reset_index()
pos_agg.columns = [
    f"{col[0]}_{col[1].upper()}" if col[1] else col[0].upper()
    for col in pos_agg.columns
]

In [13]:
pos_agg = pos_agg.pivot_table(index="SK_ID_CURR",
                    columns="NAME_CONTRACT_STATUS",
                    values=['MONTHS_BALANCE_MIN',
                           # 'MONTHS_BALANCE_MAX', 'MONTHS_BALANCE_COUNT', 'MONTHS_BALANCE_MEAN',
                           # 'CNT_INSTALMENT_MIN', 'CNT_INSTALMENT_MAX', 'CNT_INSTALMENT_MEAN',
                           # 'CNT_INSTALMENT_FUTURE_MIN', 'CNT_INSTALMENT_FUTURE_MAX',
                           # 'CNT_INSTALMENT_FUTURE_MEAN', 'SK_DPD_MIN', 'SK_DPD_MAX', 'SK_DPD_MEAN',
                           # 'SK_DPD_SUM', 'SK_DPD_DEF_MIN', 'SK_DPD_DEF_MAX', 'SK_DPD_DEF_MEAN',
                           # 'SK_DPD_DEF_SUM'
                            ],
                    aggfunc="sum",     # combine if there are duplicates
                    fill_value=0,         # fill missing with 0
            ).reset_index()

pos_agg.columns = [
    f"{col[0]}_{col[1].upper()}" if col[1] else col[0].upper()
    for col in pos_agg.columns
]

# Save data to /etl folder
pos_agg.to_csv("etl/pos_cash_agg.csv", header=True, index=False)

## 5 Repayment history data preprocessing

In [20]:
repay = pd.read_csv('data/installments_payments.csv')

repay.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [21]:
check_missing_col_coverage(repay)

DAYS_ENTRY_PAYMENT    0.021352
AMT_PAYMENT           0.021352
dtype: float64

In [22]:
# How many is repaid:
# PAYMENT_PERCENT
# PAYMENT_GAP
repay["PAYMENT_PERCENT"] = repay["AMT_PAYMENT"].div(repay["AMT_INSTALMENT"]).replace([np.inf, -np.inf], np.nan)
repay['PAYMENT_GAP'] = repay['AMT_INSTALMENT'] - repay['AMT_PAYMENT']

# Time point for repay: positive value-miss the due, negative-paid earlier
repay['PAYMENT_TIME'] = repay['DAYS_ENTRY_PAYMENT'] - repay['DAYS_INSTALMENT']

In [23]:
repay_aggmethod = {'DAYS_INSTALMENT':['min', 'max', 'mean'],
                   'DAYS_ENTRY_PAYMENT':['min', 'max', 'mean'],
                   'AMT_INSTALMENT':['min', 'max', 'mean', 'sum'],
                   'AMT_PAYMENT':['min', 'max', 'mean','sum'],
                   'PAYMENT_PERCENT':['min', 'max', 'mean'],
                   'PAYMENT_GAP':['min', 'max', 'mean','sum'],
                   'PAYMENT_TIME':['min', 'max', 'mean','sum']}

repay_agg = repay.groupby('SK_ID_CURR').agg(repay_aggmethod).reset_index()
repay_agg.columns = [col[0].upper()+'_'+col[1].upper()+"_REPAY" if col[1] else col[0].upper() for col in repay_agg.columns]

In [24]:
# Save data to /etl folder
repay_agg.to_csv("etl/installment_pay.csv", header=True, index=False)

In [25]:
repay_agg.head()

,SK_ID_CURR,DAYS_INSTALMENT_MIN_REPAY,DAYS_INSTALMENT_MAX_REPAY,DAYS_INSTALMENT_MEAN_REPAY,DAYS_ENTRY_PAYMENT_MIN_REPAY,DAYS_ENTRY_PAYMENT_MAX_REPAY,DAYS_ENTRY_PAYMENT_MEAN_REPAY,AMT_INSTALMENT_MIN_REPAY,AMT_INSTALMENT_MAX_REPAY,AMT_INSTALMENT_MEAN_REPAY,...,PAYMENT_PERCENT_MAX_REPAY,PAYMENT_PERCENT_MEAN_REPAY,PAYMENT_GAP_MIN_REPAY,PAYMENT_GAP_MAX_REPAY,PAYMENT_GAP_MEAN_REPAY,PAYMENT_GAP_SUM_REPAY,PAYMENT_TIME_MIN_REPAY,PAYMENT_TIME_MAX_REPAY,PAYMENT_TIME_MEAN_REPAY,PAYMENT_TIME_SUM_REPAY
0,100001,-2916.0,-1619.0,-2187.714286,-2916.0,-1628.0,-2195.000000,3951.000,17397.900,5885.132143,...,1.0,1.0,0.0,0.0,0.0,0.0,-36.0,11.0,-7.285714,-51.0
1,100002,-565.0,-25.0,-295.000000,-587.0,-49.0,-315.421053,9251.775,53093.745,11559.247105,...,1.0,1.0,0.0,0.0,0.0,0.0,-31.0,-12.0,-20.421053,-388.0
2,100003,-2310.0,-536.0,-1378.160000,-2324.0,-544.0,-1385.320000,6662.970,560835.360,64754.586000,...,1.0,1.0,0.0,0.0,0.0,0.0,-14.0,-1.0,-7.160000,-179.0
3,100004,-784.0,-724.0,-754.000000,-795.0,-727.0,-761.666667,5357.250,10573.965,7096.155000,...,1.0,1.0,0.0,0.0,0.0,0.0,-11.0,-3.0,-7.666667,-23.0
4,100005,-706.0,-466.0,-586.000000,-736.0,-470.0,-609.555556,4813.200,17656.245,6240.205000,...,1.0,1.0,0.0,0.0,0.0,0.0,-37.0,1.0,-23.555556,-212.0
